## 기본 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import numpy as np

## 데이터셋 준비


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# (NUM_SAMPLE, 28, 28) -> (NUM_SAMPLE, 28, 28, 1)
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 정의

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        
        self.sequence = list()
        
        # 같은 크기의 conv layer를 반복하는 것은 VGG16의 특징.
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(maxpool((2,2))) # 14x14x16
        
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(maxpool((2,2))) # 7x7x32
        
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        
        self.sequence.append(tf.keras.layers.Flatten()) # 1568
        self.sequence.append(tf.keras.layers.Dense(128, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))

    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [5]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')  # batch별 loss의 평균
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [8]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result(),
                          test_loss.result(),
                          test_accuracy.result()))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.125, Accuracy: 0.961, Test Loss: 0.040, Test Accuracy: 0.988
Epoch 2, Loss: 0.044, Accuracy: 0.987, Test Loss: 0.049, Test Accuracy: 0.986
Epoch 3, Loss: 0.031, Accuracy: 0.990, Test Loss: 0.031, Test Accuracy: 0.990
Epoch 4, Loss: 0.024, Accuracy: 0.992, Test Loss: 0.032, Test Accuracy: 0.991
Epoch 5, Loss: 0.020, Accuracy: 0.993, Test Loss: 0.031, Test Accuracy: 0.991
Epoch 6, Loss: 0.017, Accuracy: 0.995, Test Loss: 0.029, Test Accuracy: 0.991
Epoch 7, Loss: 0.016, Accuracy: 0.995, Test Loss: 0.026, Test Accuracy: 0.993
Epoch 8, Loss: 0.012, Accuracy: 0.996, Test Loss: 0.028, Test Accuracy: 0.992
Epoch 9, Loss: 0.012, Accuracy: 0.996, Test Loss: 0.028, Test Accuracy: 0.993
Epoch 10, Loss: 0.011, Accuracy: 0.997, Test Loss: 0.033, Test Accuracy: 0.992
